We install here the required dependencies

In [1]:
!python -m pip install kuzu

In [2]:
import kuzu

db = kuzu.Database('./kuzudb_elt')
conn = kuzu.Connection(db)

load_json_extension = """
INSTALL json;
LOAD EXTENSION json;
"""

conn.execute(load_json_extension)

conn.close()

conn = kuzu.Connection(db)

Let's drop the old edges and then nodes (if they exist)

In [3]:
SHOW_EDGE_TABLES = """
CALL SHOW_TABLES() WHERE type='REL' RETURN *;
"""
result_set = conn.execute(SHOW_EDGE_TABLES)
while result_set.has_next():
    table_name = result_set.get_next()[0]
    print(f"DROPPING {table_name}")
    conn.execute(f"DROP TABLE {table_name};")

result_set.close()

SHOW_NODE_TABLES = """
CALL SHOW_TABLES() WHERE type='NODE' RETURN *;
"""
result_set = conn.execute(SHOW_NODE_TABLES)
while result_set.has_next():
    table_name = result_set.get_next()[0]
    print(f"DROPPING {table_name}")
    conn.execute(f"DROP TABLE {table_name};")

result_set.close()

Now we load the nodes.

In [4]:
with open('dml_statements/elt_kuzudb_node_schema.sql') as f:
    creation_lines = [line for line in f]

with open('dml_statements/elt_kuzudb_node_import.sql') as f:
    import_lines = [line for line in f]
    

#do not forget the primary keys for seeds

    
for line in creation_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

for line in import_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CAST(cast_id STRING, roles STRING[], PRIMARY KEY (cast_id));

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CATEGORY(category_id INT64, parent_id INT64, root_id INT64, names STRUCT(name STRING, language_iso_639_1 STRING)[], PRIMARY KEY (category_id));

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CHARACTER(id INT64, name STRING, PRIMARY KEY (id));

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CONTENT(content_id INT64, date DATE, content_type STRING, runtime INT64, revenue INT64, budget INT64, homepage STRING, vote_average FLOAT, votes_count INT64, last_update TIMESTAMP, countries STRING[],  abstracts STRUCT(abstract STRING, language_iso_639_1 STRING)[], content_links STRUCT(source STRING, key STRING, language_iso_639_1 STRING)[], refs STRUCT(type STRING, referenced_id INT64)[], trailers STRUCT(source STRING, key STRING, language_iso_639_1 STRING)[], aliasings STRUCT(name STRING, language_iso_639_1 STRING)[], PRIMAR

RuntimeError: Binder exception: Primary key episode_id does not match any of the predefined node properties.

Now we load the relationships.

In [11]:
with open('dml_statements/elt_kuzudb_rel_schema.sql') as f:
    creation_lines = [line for line in f]

with open('dml_statements/elt_kuzudb_rel_import.sql') as f:
    import_lines = [line for line in f]
    

#do not forget the primary keys for seeds

    
for line in creation_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

for line in import_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

Executing statement CREATE REL TABLE OMDB_BUSINESS_CATEGORY_HAS_PARENT(FROM OMDB_BUSINESS_CATEGORY TO OMDB_BUSINESS_CATEGORY);

Executing statement CREATE REL TABLE OMDB_BUSINESS_CATEGORY_HAS_ROOT(FROM OMDB_BUSINESS_CATEGORY TO OMDB_BUSINESS_CATEGORY);

Executing statement CREATE REL TABLE OMDB_BUSINESS_EPISODE_HAS_CAST(FROM OMDB_BUSINESS_EPISODE TO OMDB_BUSINESS_CAST);

Executing statement CREATE REL TABLE OMDB_BUSINESS_EPISODE_HAS_CATEGORY(FROM OMDB_BUSINESS_EPISODE TO OMDB_BUSINESS_CATEGORY);

Executing statement CREATE REL TABLE OMDB_BUSINESS_EPISODE_HAS_KEYWORD(FROM OMDB_BUSINESS_EPISODE TO OMDB_BUSINESS_CATEGORY);

Executing statement CREATE REL TABLE OMDB_BUSINESS_EPISODE_HAS_SEASON(FROM OMDB_BUSINESS_EPISODE TO OMDB_BUSINESS_SEASON);

Executing statement CREATE REL TABLE OMDB_BUSINESS_EPISODE_HAS_SERIES(FROM OMDB_BUSINESS_EPISODE TO OMDB_BUSINESS_SERIES);

Executing statement CREATE REL TABLE OMDB_BUSINESS_MOVIE_HAS_CAST(FROM OMDB_BUSINESS_MOVIE TO OMDB_BUSINESS_CAST);

Executi

RuntimeError: Runtime exception: Found NULL, which violates the non-null constraint of the primary key column.

: 